# Bayer filter

To form a color image, we need to collect information at RGB wavelengths for all the pixels or sensors. But this process is expensive both in terms of time and money.

So in 1976, Bayer thought of an alternative. Instead of capturing all RGB information at each pixel, Bayer thought of capturing one out of RGB for each pixel. Now, each pixel will contain either R, G or B. To be able to form a color image, he decided 50% pixels be Green and rest equally to Red and Blue (to mimic human eye) and these are arranged in a pattern as shown below
![title](./assets/bayer_pattern.png)
The pattern can take different forms for example the one shown here is GRBG if the pattern is rotated the pattern will be different but it will preserve the same colors ration of Green 50%, Red 25% and Blue 25 %

An Example of bayer image is shown in the following image
![title](./assets/image_bayer.png)
After interpolation the image to RGB image
![title](./assets/image_real.png)
## Why Green is 50 %
There are as many green pixels as there are blue and red combined. This is because the human eye is not equally sensitive to all three colors. It's necessary to include more information from the green pixels in order to create an image that the eye will perceive as a "true color."

The gray scale or intensity of the color is computed as following:
$$I = 0.299R + 0.587G + 0.114B$$
## Generating RGB image
"Demosaicing" is the process of translating this Bayer array of primary colors into a final image which contains full color information at each pixel.

Check https://www.cambridgeincolour.com/tutorials/camera-sensors.htm for more explanation 

In [29]:
import cv2
import numpy as np   

In [30]:
img = cv2.imread('./assets/my_image_bayer.png', cv2.IMREAD_GRAYSCALE)
print('image shape', img.shape)
print('image range:[{}, {}] and dtype: {}'.format(img.min(), img.max(), img.dtype))
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

image shape (600, 408)
image range:[2, 255] and dtype: uint8


In [31]:
def visualize_image(figure_name: str, img: np.ndarray):
    cv2.imshow(figure_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Now convert the bayer image into RGB image using `cv2.cvtColor` function, for docs check https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html

Note the enum used for conversion `COLOR_BAYER_GR2BGR`. Since bayer pattern can take many forms, RGGB pattern is selected

In [32]:
###      MY CODE   ###
#solution :
img = cv2.imread('./assets/my_image_bayer.png', cv2.IMREAD_GRAYSCALE)
img_after_conversion=cv2.cvtColor(img,cv2.COLOR_BAYER_GR2BGR)
print('image shape', img.shape)
print('image range:[{}, {}] and dtype: {}'.format(img.min(), img.max(), img.dtype))
cv2.imshow('img_after_conversion', img_after_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

image shape (600, 408)
image range:[2, 255] and dtype: uint8


In [43]:
##Please,note there is a problem in this code 

img = cv2.imread('./assets/rgb_color_space.png',0)
i,j= img.shape 

my_rgb[i//2, j//2, 0] = img[i+1, j+1]        
my_rgb[i//2, j//2, 1] = ((int(img[i+1, j]) + int(img[i, j+1]))/2)
my_rgb[i//2, j//2, 2] = img[i, j]
visualize_image('my_rgb', my_rgb)

IndexError: index 319 is out of bounds for axis 0 with size 318

In [28]:
my_rgb.shape

NameError: name 'my_rgb' is not defined

In [6]:
rgb = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2BGR)
visualize_image('rgb', rgb)
# TODO: Implement your own bayer image to RGB function
# TODO: Implement function that convert RGB image to bayer image

In [ ]:
### MY CODE ###
#1-
#img = cv2.imread('./assets/my_image_bayer.png', cv2.IMREAD_GRAYSCALE)
#rgb_2=img*rgb

#----------
#2-
"""
def rgb_brayer(rgb_img):
    bayer_img = cv2.cvtColor(rgb_img, cv2.COLOR_BG2BGR_BAYER)
    visualize_image('bayer_imge', bayer_img)
    
    return bayer_img

rgb_img = cv2.imread('./assets/rgb_img.png', cv2.IMREAD_GRAYSCALE)


"""

# Color spaces
We will now explore the difference and the need for multiple color spaces. For an example we will try to segment the following images of rubik's cube face.
![title](./assets/rubik_dark.png)
![title](./assets/rubik_light.png)

Both images are of the same face but with different light conditions

## RGB
![title](./assets/rgb_color_space.png)
The downside of RGB is:
* mixing of chrominance ( Color related information ) and luminance ( Intensity related information ) data.
* significant perceptual non-uniformity. (check the difference between the white and blue colors through the blue channel)

## YCrCb
The YCrCb color space is derived from the RGB color space and has the following three compoenents.
* Y: The Luminance  or luma(intensity)
* Cr: How far the red component from the Luminance
* Cb: How far the blue component from the Luminance 
![title](./assets/ycrcb.png)

YCrCb separates the luminance and chrominance components into different channels helping in understanding the effect of different lighting conditions
![title](./assets/ycrcb_color_space.png)

## HSV (HSI)
HSV color space has the following three components:
* H: Hue, the dominant wavelength
* S: Saturation, purity of the color 
* V: Value (intensity)
For more details about HSV check https://www.lifewire.com/what-is-hsv-in-design-1078068
![title](./assets/hsv.png)

Best thing is that it uses only one channel to describe color (H), making it very intuitive to specify color.
![title](./assets/hsv_color_space.png)

**Watch out!** Hue's value doesn't represent a color on its own!. it represents an angle on a circle as shown in the previous figure. For example hue values as degrees looks as following:
* Red falls between 0 and 60 degrees.
* Yellow falls between 61 and 120 degrees.
* Green falls between 121-180 degrees.
* Cyan falls between 181-240 degrees.
* Blue falls between 241-300 degrees.
* Magenta falls between 301-360 degrees

### How to represent white then ?!!
White is not a color or if we are using H(Hue) terminology, white is not a wave length so how it is represented?

From the figure above we can see that White can be represented by any Hue! but only when saturation is 0. Then how to determine how white the color is? For example if the pixel `P1` has the values `(255, 255, 255)` in RGB and `P2` has `(250, 250, 250)` how can we know the difference using Hue and Saturation?

The answer is that we can't! We need to use the Value channel for this! `P1` in HSV has the value `(0, 0, 255)` and `P2` has `(0, 0, 250)`.

The same idea can be used for black. Some variations of HSV help with this problem for example HSL 

Check the following tutorial for more explanation https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/
If you want to see more color spaces check https://www.w3schools.com/colors/default.asp

In [9]:
light_img = cv2.imread('./assets/rubik_light.png')
dark_img = cv2.imread('./assets/rubik_dark.png')
visualize_image('light_img', light_img)
visualize_image('light_img', dark_img)

ycrcb_light = cv2.cvtColor(light_img, cv2.COLOR_BGR2YCrCb)
ycrcb_dark = cv2.cvtColor(dark_img, cv2.COLOR_BGR2YCrCb)

hsv_light = cv2.cvtColor(light_img, cv2.COLOR_BGR2HSV)
hsv_dark = cv2.cvtColor(dark_img, cv2.COLOR_BGR2HSV)

visualize_image('img', hsv_dark)